In [1]:
import cv2
from cv2 import aruco
import numpy as np
import msgpack as mp
import msgpack_numpy as mpn
import os
import matplotlib.pyplot as plt

In [2]:
_webcam_calib_video = r"D:\CMC\pyprojects\visual-vibe\calibration\webcam_color.msgpack"

In [3]:
ARUCO_PARAMETERS = aruco.DetectorParameters()
ARUCO_DICT = aruco.getPredefinedDictionary(aruco.DICT_ARUCO_ORIGINAL)
detector = aruco.ArucoDetector(ARUCO_DICT, ARUCO_PARAMETERS)
markerLength = 0.05
markerSeperation = 0.01

board = aruco.GridBoard(
        size= [1,1],
        markerLength=markerLength,
        markerSeparation=markerSeperation,
        dictionary=ARUCO_DICT)

In [4]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)
_video_length = 0

for _frame in _video_data:
    corners, ids, rejected_image_points = detector.detectMarkers(_frame)
    if ids is not None:

        _video_length += 1

_video_file.close()

In [5]:
_video_pth = _webcam_calib_video
_video_file = open(_video_pth, "rb")
_video_data = mp.Unpacker(_video_file, object_hook=mpn.decode)

marker_corners = []
object_points = []
first = True
counter = []
for idx, _frame in enumerate(_video_data):
    
    _frame = cv2.cvtColor(_frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejected_image_points = detector.detectMarkers(_frame)
    if ids is not None:
        if first == True:
            corners_list = corners
            id_list = ids
            first = False
        else:
            corners_list = np.vstack((corners_list, corners))
            id_list = np.vstack((id_list,ids))
        counter.append(len(ids))

_video_file.close()

counter = np.array(counter)

In [6]:
rnd = np.random.choice(_video_length, 150, replace=False)

In [7]:
rnd.sort()
rnd

array([  2,   5,   6,  14,  16,  17,  20,  23,  26,  31,  55,  56,  58,
        59,  60,  68,  70,  83,  86,  95, 101, 105, 106, 113, 114, 118,
       121, 128, 135, 141, 147, 155, 167, 177, 179, 185, 187, 196, 199,
       205, 209, 219, 223, 228, 240, 242, 243, 248, 255, 259, 261, 270,
       277, 279, 280, 291, 292, 299, 301, 302, 309, 311, 313, 314, 319,
       322, 323, 325, 334, 342, 352, 354, 360, 371, 372, 374, 376, 381,
       390, 393, 399, 402, 404, 408, 409, 418, 422, 429, 434, 450, 452,
       460, 463, 466, 482, 483, 485, 490, 491, 508, 509, 513, 516, 521,
       526, 532, 541, 547, 562, 570, 572, 580, 581, 587, 594, 602, 607,
       618, 626, 635, 637, 639, 642, 645, 649, 650, 652, 655, 658, 663,
       673, 677, 680, 685, 693, 695, 696, 701, 702, 705, 721, 752, 754,
       756, 757, 758, 760, 770, 771, 772])

In [38]:
arr = np.ones(len(corners_list), dtype=bool)
current_index = 0
previous_index = 0
for idx, vals in enumerate(counter):
    previous_index = current_index
    current_index += vals
    if idx not in rnd:
        arr[previous_index:current_index] = False

selected_corners = corners_list[arr, ...]
selected_ids = id_list[arr, ...]
selected_counter = counter[rnd]

In [11]:
rnd.sort()
rnd

array([  2,   5,   6,  14,  16,  17,  20,  23,  26,  31,  55,  56,  58,
        59,  60,  68,  70,  83,  86,  95, 101, 105, 106, 113, 114, 118,
       121, 128, 135, 141, 147, 155, 167, 177, 179, 185, 187, 196, 199,
       205, 209, 219, 223, 228, 240, 242, 243, 248, 255, 259, 261, 270,
       277, 279, 280, 291, 292, 299, 301, 302, 309, 311, 313, 314, 319,
       322, 323, 325, 334, 342, 352, 354, 360, 371, 372, 374, 376, 381,
       390, 393, 399, 402, 404, 408, 409, 418, 422, 429, 434, 450, 452,
       460, 463, 466, 482, 483, 485, 490, 491, 508, 509, 513, 516, 521,
       526, 532, 541, 547, 562, 570, 572, 580, 581, 587, 594, 602, 607,
       618, 626, 635, 637, 639, 642, 645, 649, 650, 652, 655, 658, 663,
       673, 677, 680, 685, 693, 695, 696, 701, 702, 705, 721, 752, 754,
       756, 757, 758, 760, 770, 771, 772])

In [12]:
calibration_flags = cv2.CALIB_USE_LU
term_criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6)

In [13]:
mtx2 = np.zeros((3, 3))
dist2 = np.zeros((1, 8))
rvec2 = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(len(counter))]
tvec2 = [np.zeros((1, 1, 3), dtype=np.float64) for i in range(len(counter))]

In [14]:
# ret1, mtx1, dist1, rvecs1, tvecs1 = aruco.calibrateCameraAruco(corners_list, id_list, counter, board, _frame.shape[:2], mtx2, dist2, flags=calibration_flags, criteria = term_criteria)
ret1, mtx1, dist1, rvecs1, tvecs1 = aruco.calibrateCameraAruco(selected_corners, selected_ids, selected_counter, board, _frame.shape[:2], mtx2, dist2, flags=calibration_flags, criteria = term_criteria)

In [29]:
import toml
_toml_pth = r"D:\CMC\pyprojects\visual-vibe\calibration\camera_calibration.toml"

data = toml.load(_toml_pth)
data['calibration']['camera_matrix'] = mtx1.tolist()
data['calibration']['dist_coeffs'] = dist1.tolist()
with open(_toml_pth, 'w') as f:
    toml.dump(data, f)

In [15]:
mtx1

array([[1.55197429e+03, 0.00000000e+00, 3.85140807e+02],
       [0.00000000e+00, 1.96280120e+03, 6.19299916e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])